# Imports

In [1]:
# all imports

import pandas as pd

# Creating All Dataframes (5 Minutes to Run)

In [2]:
# creating all dataframes

df_titles = pd.read_csv('title.basics.tsv', sep='\t')

/Users/sachilad/anaconda3/envs/sachi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_principals = pd.read_csv('title.principals.tsv', sep='\t')

In [4]:
df_actors = pd.read_csv('name.basics.tsv', sep='\t')

# Cleaning Dataframes

In [81]:
# cleaning df_titles

df_titles_clean = df_titles.copy()

# remove everything but movies
df_titles_clean_0 = df_titles_clean[df_titles_clean['titleType'] == 'movie']

# remove adult titles
df_titles_clean_1 = df_titles_clean_0[df_titles_clean_0['isAdult'] != 1]

# remove start year being '\N'
df_titles_clean_2 = df_titles_clean_1[df_titles_clean_1['startYear'] != '\\N']

# remove movies older than 1925 (assuming no actor is older than 100)
df_titles_clean_2['startYear'] = pd.to_numeric(df_titles_clean_2['startYear'], errors='coerce')
df_titles_clean_3 = df_titles_clean_2[df_titles_clean_2['startYear'] >= 1925]

df_titles_final = df_titles_clean_3.copy()

# print(df_titles_final.head())

/Users/sachilad/anaconda3/envs/sachi/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


# Manipulating Dataframes for MCU

### Filtering to Just MCU Movies

In [82]:
'''filtering to just mcu movies'''

# remove movies older than 2008 (as that was the first mcu release movie)
df_titles_final = df_titles_final[df_titles_final['startYear'] >= 2008]

# Load the CSV file containing the filter values
mcu_movies_list = pd.read_csv('mcu_movies.csv')

# Filter the df_titles DataFrame based on the values from the CSV file
df_mcu_movies = df_titles_final[df_titles_final['primaryTitle'].isin(mcu_movies_list['Movie Title'])]

# # Remove duplicates based on the originalTitle column
df_mcu_movies = df_mcu_movies[df_mcu_movies['primaryTitle'] == df_mcu_movies['originalTitle']]
# remove black widow movie duplicates
df_mcu_movies = df_mcu_movies[~((df_mcu_movies['primaryTitle'] == 'Black Widow') & (df_mcu_movies['startYear'] != 2021))]

'''
certain movies not added
* Ant-Man and the Wasp
* Ant-Man Quantumania 
'''

# # sanity check
# # Count the number of rows in the DataFrame
# num_rows = df_mcu_movies.shape[0]

# # Print the number of rows
# print(f"Number of rows: {num_rows}")

# print(df_mcu_movies)

'\ncertain movies not added\n* Ant-Man and the Wasp\n* Ant-Man Quantumania \n'

In [83]:
# get id of each movie into csv file
# tconst_series = df_mcu_movies['tconst']
# tconst_series.to_csv('id_mcu_movies.csv', index=False)

mcu_movies_ids_titles = df_mcu_movies[['tconst', 'primaryTitle']]
mcu_movies_ids_titles.columns = ['tconst', 'title']
mcu_movies_ids_titles.to_csv('id_mcu_movies.csv', index=False)


### filtering to just mcu actors

In [84]:
# filtering df_principals to get actors and actresses 

'''
# Unique values of category 
unique_values = df_principals['category'].unique()
print(unique_values)
# ['self' 'director' 'producer' 'cinematographer' 'composer' 'editor'
# 'actor' 'actress' 'writer' 'production_designer' 'archive_footage'
# 'casting_director' 'archive_sound']
'''

df_principals_clean = df_principals[df_principals['category'].isin(['actor', 'actress'])]

# filter dataframe using title from id_mcu_movies csv file
id_mcu_movies = pd.read_csv('id_mcu_movies.csv')
df_principals_mcu_actors = df_principals_clean[df_principals_clean['tconst'].isin(id_mcu_movies['tconst'])]
df_principals_mcu_actors = df_principals_mcu_actors.drop_duplicates()
print(df_principals_mcu_actors.head())

            tconst  ordering     nconst category job              characters
4139484  tt0371746         1  nm0000375    actor  \N          ["Tony Stark"]
4139485  tt0371746         2  nm0000569  actress  \N        ["Pepper Potts"]
4139486  tt0371746         3  nm0005024    actor  \N              ["Rhodey"]
4139487  tt0371746         4  nm0000313    actor  \N       ["Obadiah Stane"]
4139488  tt0371746         5  nm0004753  actress  \N  ["Christine Everhart"]


In [85]:
# create csv file of all actor ids of cast of the mcu movies
mcu_actors_ids = df_principals_mcu_actors['nconst']
mcu_actors_ids.to_csv('id_mcu_actors.csv', index=False)

In [86]:
# create csv file of actors, movies and titles

df_mcu_movie_actor_merge = df_principals_mcu_actors.merge(df_mcu_movies[['tconst', 'primaryTitle']], on='tconst', how='left')
df_mcu_ids = df_mcu_movie_actor_merge[['nconst', 'tconst', 'primaryTitle']]
df_mcu_ids = df_mcu_ids.rename(columns={'primaryTitle': 'title'})
df_mcu_ids.to_csv('mcu_network.csv', index=False)

### get mcu actor names into the csv file

In [87]:
# filtering df_actors to get only the mcu actors in the data frame

mcu_actors_ids = df_principals_mcu_actors['nconst']
df_actors_mcu = df_actors[df_actors['nconst'].isin(mcu_actors_ids)]
df_actors_mcu = df_actors_mcu.drop_duplicates()

In [88]:
# adding the actors names into the id_mcu_actors csv file

id_mcu_actors = pd.read_csv('id_mcu_actors.csv')
df_actors_mcu_with_names = id_mcu_actors.merge(df_actors_mcu[['nconst', 'primaryName']], on='nconst', how='left')
df_actors_mcu_with_names = df_actors_mcu_with_names.rename(columns={'primaryName' : 'name'})
df_actors_mcu_with_names.to_csv('id_mcu_actors.csv', index=False)

In [89]:
# adding the actors names to the mcu_network csv file

df_mcu_network = pd.read_csv('mcu_network.csv')
df_mcu_network_with_names = df_mcu_network.merge(df_actors_mcu[['nconst', 'primaryName']], on='nconst', how='left')
df_mcu_network_with_names = df_mcu_network_with_names.rename(columns={'primaryName' : 'name'})
df_mcu_network_with_names = df_mcu_network_with_names.drop_duplicates()
df_mcu_network_with_names.to_csv('mcu_network.csv', index=False)

# Manipulating Dataframes for Non-MCU Movies

In [90]:
# getting all movies done by mcu actors 

df_mcu_actors = pd.read_csv('id_mcu_actors.csv')
df_nonmcu_movies_noname = df_principals_clean[df_principals_clean['nconst'].isin(id_mcu_actors['nconst'])]

# cleaning up dataframe
columns_to_drop = ['ordering', 'job']
df_nonmcu_movies_noname = df_nonmcu_movies_noname.drop(columns=columns_to_drop)

In [91]:
num_rows = df_nonmcu_movies_noname.shape[0]
print("number of rows: ",  num_rows)

print(df_nonmcu_movies_noname)

number of rows:  27621
             tconst     nconst category             characters
418563    tt0035423  nm0413168    actor            ["Leopold"]
697612    tt0053545  nm0000602    actor         ["Tad Dundee"]
697613    tt0053545  nm0000602    actor            ["Torsett"]
747945    tt0056675  nm0000602    actor    ["Pvt. Roy Loomis"]
749755    tt0056791  nm0000621    actor  ["Jaimie McPheeters"]
...             ...        ...      ...                    ...
90103156  tt9903408  nm0001838  actress                     \N
90107236  tt9904522  nm0915989    actor           ["Narrator"]
90107237  tt9904522  nm0915989    actor    ["Self - Narrator"]
90107616  tt9904670  nm0000353    actor           ["Narrator"]
90126633  tt9909526  nm0169806    actor     ["Bodil de Resny"]

[27621 rows x 4 columns]


In [93]:
# add actor names to dataframe
df_nonmcu_movies = df_nonmcu_movies_noname.merge(df_mcu_actors[['nconst', 'name']], on='nconst', how='left')

In [100]:
# add movie title to dataframe

df_nonmcu_movies_with_titles = df_nonmcu_movies.merge(df_titles_final[['tconst', 'primaryTitle']], on='tconst', how='left')


In [ ]:
# final cleaning and save to csv 

# Drop rows where primaryTitle is NaN
df_nonmcu_movies_with_titles_cleaned = df_nonmcu_movies_with_titles.dropna(subset=['primaryTitle'])
df_nonmcu_movies_with_titles_cleaned = df_nonmcu_movies_with_titles_cleaned.drop_duplicates()
df_nonmcu_movies_with_titles_cleaned.to_csv('nonmcu_network.csv', index=False)